สวัสดีครับ สำหรับ workshop ของ [ThAIKeras](https://thaikeras.com/) ในครั้งนี้ จะเป็นการทดลองใช้ deep learning มาจำแนกหมวดหมู่ของงานศิลปะในพิพิธภัณฑ์ [The Metropolitan Museum of Art](https://www.metmuseum.org/) (หรือ the Met) โดยพิพิธภัณฑ์นี้เป็นพิพิธภัณฑ์ศิลปะที่ใหญ่ที่สุดในอเมริกา และมีผู้เข้าชมเป็นอันดับสามของโลก รองจาก Musée du Louvre และ National Museum of China เลยครับ

<table width="100%" border="0" cellspacing="0" cellpadding="0">
	<tr>        
        <td style="width:31%"><img src="https://i.imgur.com/CjpiU69.png" alt="the Met"></td>
        <td style="width:69%"><img src="https://i.imgur.com/P6sPZRk.png" alt="Arts"></td>
    </tr>
</table>

<center>(the Met และตัวอย่างงานศิลปะอันเลื่องชื่อที่จัดแสดงภายในพิพิธภัณฑ์)</center>

ด้วยความที่เป็นพิพิธภัณฑ์ขนาดใหญ่ จึงทำให้มีของสะสมอยู่เป็นจำนวนมาก ประมาณ 1.5 ล้านชิ้น ในจำนวนนี้มีอยู่สองแสนชิ้นที่ได้ทำการถ่ายภาพ และจัดหมวดหมู่เรียบร้อยแล้ว ในการจัดหมวดหมู่ของงานศิลปะ ถ้าสามารถใช้ AI เข้ามาช่วย ก็จะลดเวลาและความผิดพลาดจากการทำงานของมนุษย์ลงได้ จึงเป็นที่มาของการแข่งขัน [iMet Collection 2019](https://www.kaggle.com/c/imet-2019-fgvc6/) ซึ่งเป็นส่วนนึงของ [FGVC6 workshop](https://sites.google.com/view/fgvc6/) ในงาน [CVPR conference](http://cvpr2019.thecvf.com/) ปีนี้

<img src="https://i.imgur.com/KtKv0Nr.png" alt="logo">
<center>(โลโกของการแข่งขัน iMet Collection 2019)</center>

สำหรับ notebook นี้ จะแบ่งเป็นสองส่วนใหญ่ๆ โดยในส่วนแรกเป็นการแนะนำถึงข้อมูลที่ใช้ และทำการจัดเตรียมข้อมูล ก่อนจะไปถึงส่วนที่สอง ซึ่งเราจะสร้างโมเดลเพื่อจำแนกประเภทของงานศิลปะกันครับ

<hr>

# Data

In [ ]:
import numpy as np
import pandas as pd

import random
from tqdm import tqdm
from pathlib import Path

import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split

import itertools
from scipy.sparse import lil_matrix, coo_matrix
from collections import defaultdict, Counter

from sklearn.metrics import fbeta_score
from albumentations import (OneOf, Compose, HorizontalFlip, RandomCrop, 
                            RandomBrightness, RandomContrast, 
                            ShiftScaleRotate, IAAAdditiveGaussianNoise)

import keras
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Concatenate, Lambda, Layer

ในขั้นแรก เราจะมาดูหมวดหมู่ทั้งหมดกันก่อน โดยงานนี้เราจะต้องจำแนกว่าภาพต่างๆ จัดอยู่ในหมวดหมู่ใดได้บ้าง โดยแต่ละภาพก็สามารถจัดอยู่ในหลายหมวดหมู่ได้ ดังนั้นปัญหานี้จึงถือว่าเป็น multi-label classification นอกจากนี้ก็ยังนับว่าเป็น fine-grained visual recognition นั่นคือในการจะแยกแยะเป็นหมวดหมู่ต่างๆ ได้นั้น จำเป็นต้องลงลึกไปถึงรายละเอียดที่อยู่ในรูปภาพด้วย

ในงานนี้มีอยู่ทั้งหมด 1103 หมวดหมู่ โดยแบ่งออกเป็นสองส่วน คือ culture ซึ่งบอกว่างานศิลป์ชิ้นนี้มาจากวัฒนธรรมไหน และ tag ที่บอกว่างานศิลป์ชิ้นนี้เกี่ยวข้องกับอะไรบ้าง

In [ ]:
DATA_ROOT = Path('../input/imet-2019-fgvc6/')

label_df = pd.read_csv(DATA_ROOT/'labels.csv')
print(f"Number of attributes = {len(label_df)}")

culture_df = label_df[label_df['attribute_name'].str.startswith('culture')]
print(f"Number of cultures = {len(culture_df)}")

tag_df = label_df[label_df['attribute_name'].str.startswith('tag')]
print(f"Number of tags = {len(tag_df)}")

ไฟล์ labels.csv จะบอกหมวดหมู่ที่มีทั้งหมด หลังจากที่อ่านไฟล์มาแล้ว เราลองดูตัวอย่างหมวดหมู่ต่างๆ กันครับ

In [ ]:
label_df.sample(n=25).sort_values('attribute_id')

ด้านล่างนี้เป็นการอ่านข้อมูล และแสดงตัวอย่าง ซึ่งจะประกอบด้วยรูปภาพของงานศิลป์ กับหมวดหมู่ของงานศิลป์ชิ้นนั้นๆ

In [ ]:
data_df = pd.read_csv(DATA_ROOT/'train.csv')
data_df['attribute_ids'] = data_df['attribute_ids'].str.split().map(lambda x_list: [int(x) for x in x_list])

In [ ]:
sample_df = data_df.sample(n=10)

for _, row in sample_df.iterrows():
    img = cv2.imread(str(DATA_ROOT / 'train' / (row['id']+'.png')))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    labels = '\n'.join([label_df.loc[label]['attribute_name'] for label in row['attribute_ids']])

    _, axs = plt.subplots(1, 2, figsize=(12, 4))
        
    axs[0].imshow(img)
    axs[0].title.set_text(row['id'])
    
    axs[1].text(0, 0.5, labels, fontsize=12, ha='left', va='center')
    axs[1].axis('off')
    
    plt.show()

ต่อมาเราจะทำการแบ่งข้อมูลทั้งหมด ออกเป็น training data และ validation data โดยควรจะแบ่งให้การกระจายของคลาสมีค่าเหมือนๆ กัน ในข้อมูลทั้งสองชุด และเนื่องจากข้อมูลของเรามีลักษณะเป็น multi label การแบ่งข้อมูลจะใช้วิธี [iterative stratification](http://lpis.csd.auth.gr/publications/sechidis-ecmlpkdd-2011.pdf)

แต่ทว่าเมื่อแบ่งข้อมูลแบบนี้ โดยลองใช้โค้ดของ [scikit-mulitlearn](http://scikit.ml/) หรือ[โค้ดของคุณ Lopuhin](https://github.com/lopuhin/kaggle-imet-2019/blob/master/imet/make_folds.py) แล้วพบว่าการกระจายของคลาสในข้อมูลทั้งสองชุดไม่ค่อยจะเหมือนกัน อย่างไรก็ตาม เมื่อนำไปใช้งาน ค่าความถูกต้องจาก validation data และค่าความถูกต้องของ test data ก็ยังสอดคล้องกันอยู่ครับ

In [ ]:
# train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=42)

In [ ]:
# attributes = data_df['attribute_ids'].tolist()

# row = list(itertools.chain(*[[i]*len(attributes[i]) for i in range(len(data_df))]))
# col = list(itertools.chain(*attributes))

# y = coo_matrix(([1]*len(row), (row, col)), shape=(len(data_df), len(label_df))).tolil()

# train_df, _, valid_df, _ = iterative_train_test_split(data_df.values, y, test_size=0.2)
# train_df = pd.DataFrame(train_df, columns=['id', 'attribute_ids'])
# valid_df = pd.DataFrame(valid_df, columns=['id', 'attribute_ids'])

In [ ]:
def make_folds(n_folds: int) -> pd.DataFrame:
    df = pd.read_csv(DATA_ROOT / 'train.csv')
    cls_counts = Counter(cls for classes in df['attribute_ids'].str.split() for cls in classes)
    fold_cls_counts = defaultdict(int)
    folds = [-1] * len(df)
    for item in tqdm(df.sample(frac=1, random_state=42).itertuples(), total=len(df)):
        cls = min(item.attribute_ids.split(), key=lambda cls: cls_counts[cls])
        fold_counts = [(f, fold_cls_counts[f, cls]) for f in range(n_folds)]
        min_count = min([count for _, count in fold_counts])
        random.seed(item.Index)
        fold = random.choice([f for f, count in fold_counts if count == min_count])
        folds[item.Index] = fold
        for cls in item.attribute_ids.split():
            fold_cls_counts[fold, cls] += 1
    df['fold'] = folds
    return df

fold_df = make_folds(5)
fold_df['attribute_ids'] = fold_df['attribute_ids'].str.split().map(lambda x_list: [int(x) for x in x_list])

train_df = fold_df[fold_df['fold'] != 0].reset_index(drop=True)
valid_df = fold_df[fold_df['fold'] == 0].reset_index(drop=True)

In [ ]:
train_attributes = list(itertools.chain(*train_df['attribute_ids'].tolist()))
print("Total train images: ", len(train_df))
print("Total train attributes: ", len(train_attributes))

plt.figure(figsize=(12, 3))
values, counts = np.unique(train_attributes, return_counts=True)
plt.bar(values, counts)
plt.show()

valid_attributes = list(itertools.chain(*valid_df['attribute_ids'].tolist()))
print("Total validation images: ", len(valid_df))
print("Total validation attributes: ", len(valid_attributes))

plt.figure(figsize=(12, 3))
values, counts = np.unique(valid_attributes, return_counts=True)
plt.bar(values, counts)
plt.show()

# Model

ในที่นี้ เราจะนำโมเดล [EfficientNet](https://arxiv.org/abs/1905.11946) ซึ่งเป็น SOTA ทาง computer vision ในปัจจุบันมาใช้งาน โดยเริ่มจากการโหลดโค้ด EfficientNet ที่เป็นเวอร์ชัน Keras มาครับ

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.keras import EfficientNetB3, preprocess_input

In [ ]:
EPOCHS = 8
BATCH_SIZE = 32

INPUT_SHAPE = (288, 288, 3)
NUM_CLASS = len(label_df)

รูปภาพที่ส่งเป็น input ให้กับโมเดลจะมีการทำ image augmentation ก่อน สำหรับเรื่อง image augmentation สามารถศึกษาเพิ่มเติมได้จากอีก [workshop ของเรา](https://www.kaggle.com/ratthachat/workshop-augmentation-image-ai-for-eyes-2)

In [ ]:
def augment(p=1.0):
    return Compose([
        HorizontalFlip(p=0.5),
#         OneOf([
#             RandomBrightness(0.1, p=1.0),
#             RandomContrast(0.1, p=1.0),
#         ], p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
#         IAAAdditiveGaussianNoise(p=0.3),      
        RandomCrop(INPUT_SHAPE[0], INPUT_SHAPE[1])
    ], p=p)

ส่วน input ที่ป้อนให้กับโมเดล จะสร้างด้วย generator ซึ่งในที่นี้ทำโดยใช้คลาสที่เป็น extension ของ `keras.utils.Sequence` และมีฟังก์ชัน `__getitem__` เป็นหลักที่จะส่ง minibatch กลับมา สำหรับรายละเอียดของการสร้าง generator เพิ่มเติม สามารถอ่านได้[ที่นี่](https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly)ครับ

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size, shuffle=True):
        self.df = df        
        self.indices = np.arange(len(self.df))
        
        self.batch_size = batch_size        
        self.shuffle = shuffle
        
        if self.shuffle:
            np.random.shuffle(self.indices)

        self.path = DATA_ROOT / 'train'

    def __len__(self):
        return int(np.ceil(len(self.df)/self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size: (idx+1)*self.batch_size]        
        
        batch_images = np.zeros((len(batch_indices), *INPUT_SHAPE))
        batch_labels = np.zeros((len(batch_indices), NUM_CLASS))
        
        for i in range(len(batch_indices)):
            row = self.df.iloc[batch_indices[i]]
            
            path = self.path / (row['id']+'.png')
            img = cv2.cvtColor(cv2.imread(str(path)), cv2.COLOR_BGR2RGB)
        
            img = augment()(image=img)['image']
            batch_images[i] = preprocess_input(img)
            
            for label in row['attribute_ids']:
                batch_labels[i][label] = 1
                
        batch_images = np.array(batch_images, np.float32)
        return batch_images, batch_labels
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [ ]:
train_generator = DataGenerator(train_df, BATCH_SIZE)
valid_generator = DataGenerator(valid_df, BATCH_SIZE, shuffle=False)

ในส่วนนี้จะเป็นการสร้างโมเดล EfficientNet แบบ B3 โดยมี pretrained weight จาก ImageNet และทำ global average pooling ต่อจาก convolutional layer ชั้นสุดท้าย แล้วเพิ่ม dense layer ชั้นนึง ก่อนที่จะเป็นชั้น output ที่ใช้ sigmoid activation จำนวนเท่ากับหมวดหมู่ที่มีครับ

In [ ]:
def create_model(input_shape):
    base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
    input_tensor = base_model.input
    
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu',name='final_features')(x)
    x = Dropout(0.5)(x)
    output = Dense(NUM_CLASS, activation='sigmoid')(x)

    model = Model(input_tensor, output)
    return model

model = create_model(INPUT_SHAPE)
model.summary()

ในการแข่งขันครั้งนี้ วัดผลโดยใช้ [F2 score](https://en.wikipedia.org/wiki/F1_score#Definition) เราจึงสร้างคลาส `F2Evaluation` เป็น callback ของ Keras เพื่อสังเกตการณ์ค่า F2 หลังจากเสร็จสิ้นการรันทุกรอบ

เนื่องจาก output จากโมเดลเป็น probability ของหมวดหมู่ต่างๆ จึงต้องกำหนด threshold ไว้ว่าค่า probability สูงเกินกว่าเท่าไหร่ จึงจะตอบหมวดหมู่นี้ออกมา โดยได้ทดลองใช้ค่า threshold ตั้งแต่ 0.05 ถึง 0.5 ดูว่าค่า threshold ไหน จะให้ความถูกต้องสูงสุด และพบว่าค่า threshold เท่ากับ 0.1 ดีที่สุดครับ

นอกจากนี้ จะใส่เงื่อนไขเพิ่มเติมว่าไม่ให้ผลลัพธ์ออกมาเกิน 10 หมวดหมู่ และอย่างน้อยให้ตอบออกมา 1 หมวดหมู่เสมอ โดยกระบวนการสร้างคำตอบจาก output ของโมเดล จะอยู่ในฟังก์ชัน `binarize_prediction`

In [ ]:
train_f2_hist = []
valid_f2_hist = []

def generate_labels(df):
    labels = np.zeros((len(df), NUM_CLASS))

    for i, row in df.iterrows():
        for label in row['attribute_ids']:
            labels[i][label] = 1
            
    return labels

def _make_mask(argsorted, top_n: int):
    mask = np.zeros_like(argsorted, dtype=np.uint8)
    col_indices = argsorted[:, -top_n:].reshape(-1)
    row_indices = [i // top_n for i in range(len(col_indices))]
    mask[row_indices, col_indices] = 1
    return mask

def binarize_prediction(predictions, threshold: float, min_labels=1, max_labels=10):
    assert predictions.shape[1] == NUM_CLASS
    argsorted = predictions.argsort(axis=1)
    max_mask = _make_mask(argsorted, max_labels)
    min_mask = _make_mask(argsorted, min_labels)
    prob_mask = predictions > threshold
    return (max_mask & prob_mask) | min_mask

class F2Evaluation(Callback):
    def __init__(self, interval=1):
        super(Callback, self).__init__()

        self.interval = interval        
        self.train_generator = DataGenerator(train_df, BATCH_SIZE, shuffle=False)
        
        self.train_y = generate_labels(train_df)
        self.valid_y = generate_labels(valid_df)        
        
    def predict(self, generator, y_true):
        predictions = self.model.predict_generator(generator, verbose=1)
        
        best_threshold = 0.0
        best_score = 0.0
        
        for threshold in np.arange(0.05, 0.55, 0.05):
            #y_pred = np.where(predictions > threshold, 1, 0)
            y_pred = binarize_prediction(predictions, threshold)
            
            f2_score = fbeta_score(y_true, y_pred, beta=2, average='samples')
            
            if f2_score > best_score:
                best_score = f2_score
                best_threshold = threshold
            
        return best_score, best_threshold        

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval != 0:
            return
        
        train_f2_score, train_threshold = self.predict(self.train_generator, self.train_y)
        valid_f2_score, valid_threshold = self.predict(valid_generator, self.valid_y)
        
        train_f2_hist.append(train_f2_score)             
        print("train f2 = %.4f (threshold = %.2f)" % (train_f2_score, train_threshold))        

        valid_f2_hist.append(valid_f2_score)             
        print("valid f2 = %.4f (threshold = %.2f)" % (valid_f2_score, valid_threshold))

        if valid_f2_score >= max(valid_f2_hist):
            print('save checkpoint: ', valid_f2_score)
            self.model.save_weights('model_bestf2.h5')

f2_metric = F2Evaluation(interval=1)

เมื่อถึงตอนนี้ก็จะเริ่มเทรนโมเดลได้ โดยให้ loss เป็น `binary_crossentropy` ซึ่งเหมาะกับปัญหา multi-label classification ส่วน optimizer ก็ใช้เป็น [Adam](https://arxiv.org/abs/1412.6980) ปกติครับ

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(3e-4))

hist = model.fit_generator(train_generator, 
                           validation_data=valid_generator, 
                           epochs=EPOCHS, verbose=1,
                           callbacks=[f2_metric], 
                           use_multiprocessing=True, workers=2)

In [ ]:
plt.plot(range(1, EPOCHS+1), hist.history['loss'], label='train_loss')
plt.plot(range(1, EPOCHS+1), hist.history['val_loss'], label='valid_loss')
plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

plt.plot(range(1, EPOCHS+1), train_f2_hist, label='train_f2')
plt.plot(range(1, EPOCHS+1), valid_f2_hist, label='valid_f2')
plt.legend()
plt.ylabel('f2')
plt.xlabel('epoch')
plt.show()

เมื่อเทรนเสร็จแล้ว เราจะมาลองทดสอบโมเดลที่ได้ โดยมีการทำ tta (test-time augmentation) คือทำการทำนายหลายครั้ง ด้วย augmentation ที่แตกต่างกัน แล้วเอาผลลัพธ์ที่ได้มาเฉลี่ย วิธีนี้จะทำให้มีความถูกต้องสูงขึ้น

In [ ]:
model.load_weights('model_bestf2.h5')
predictions = np.zeros((len(valid_df), NUM_CLASS))

for _ in range(4):
    predictions += model.predict_generator(valid_generator, verbose=1)
predictions /= 4

y_true = generate_labels(valid_df)
y_pred = binarize_prediction(predictions, 0.1)

valid_f2_score = fbeta_score(y_true, y_pred, beta=2, average='samples')
print("valid tta f2 = %.4f (threshold = 0.10)" % valid_f2_score)

สุดท้ายนี้ เรามาดูตัวอย่างภาพใน validation set พร้อมหมวดหมู่จริงเปรียบเทียบกับหมวดหมู่ที่โมเดลทำนายได้กันครับ

In [ ]:
sample_df = valid_df.sample(n=10)

for i, row in sample_df.iterrows():
    img = cv2.imread(str(DATA_ROOT / 'train' / (row['id']+'.png')))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.title(row['id'])
    plt.imshow(img)
    plt.show()

    true_labels = ', '.join([label_df.loc[label]['attribute_name'] 
                             for label in row['attribute_ids']])
    
    pred_labels = ', '.join([label_df.loc[label]['attribute_name'] 
                             for label in np.where(y_pred[i]==1)[0]])
    
    print("True labels = " + true_labels)
    print("Predicted labels = " + pred_labels)

<hr>

จะเห็นว่าปัญหานี้มีความยากอยู่พอสมควร ในการทายหมวดหมู่ให้ถูกต้องตรงกับที่ผู้เชี่ยวชาญกำหนดมา ทั้งนี้เป็นเพราะหลายสาเหตุด้วยกัน อาทิเช่น
* บางหมวดหมู่ค่อนข้างจะเฉพาะเจาะจง และมีตัวอย่างใน training data น้อย
* ส่วน culture ในบางภาพอาจไม่สามารถดูออกได้อย่างชัดเจน และส่วน tag ในบางภาพต้องสังเกตรายละเอียดเล็กๆ น้อยๆ ให้ดี จึงจะเห็นว่าเป็นหมวดหมู่นั้น
* บางครั้งคำตอบที่โมเดลทำนายมาก็ถือว่าถูกต้องแล้ว แต่ใน label จริงตกหล่นไป (การแข่งขันครั้งนี้วัดผลด้วย F2 score ที่เน้น recall มากกว่า ถ้าโมเดลตอบเกินมาจะดีกว่าตอบขาดไปครับ)

โค้ดนี้เป็นเพียงการสาธิตเบื้องต้น ยังสามารถปรับปรุงให้ความถูกต้องสูงขึ้นได้อีก ท่านที่สนใจอาจศึกษาเพิ่มเติมจากวิธีการของผู้เข้าแข่งขันที่ได้อันดับสูง ในหน้า [discussion](https://www.kaggle.com/c/imet-2019-fgvc6/discussion) หรือจาก[เปเปอร์ที่เกี่ยวข้อง](https://paperswithcode.com/task/fine-grained-image-classification)

สำหรับการแข่งขัน iMet Collection 2019 นี้ ทาง ThAIKeras ก็ได้เข้าร่วม และสามารถคว้าเหรียญเงินมาครอง ในเวลานั้นเราได้ใช้[โค้ด](https://www.kaggle.com/lopuhin/imet-2019-submission)ชั้นดีของคุณ [Konstantin Lopuhin](https://www.kaggle.com/lopuhin) เป็นจุดตั้งต้น ซึ่งตอนนี้คุณ Lopuhin ก็ได้เป็น grandmaster แล้ว ต้องขอขอบคุณและแสดงความยินดีไว้ ณ ที่นี้ด้วยครับ

ท่านผู้อ่านสามารถนำ notebook นี้ ไปรันเล่นหรือแก้ไขเพิ่มเติมได้ หากพบข้อผิดพลาดหรือมีข้อสงสัยตรงไหน สามารถบอกมาได้ที่ด้านล่างนี้ หรือที่[เว็บบอร์ด](https://thaikeras.com/community/)ของ ThAIKeras ก็ได้ครับ